# オンライン特徴量ストアのサンプルノートブック

このノートブックは、Databricks Feature Storeを使用して、リアルタイムサービングと自動特徴量ルックアップのためにDatabricksオンラインテーブルに特徴量を公開する方法を示しています。問題は、さまざまな静的なワインの特徴量とリアルタイム入力を使用して、MLモデルでワインの品質を予測することです。

このノートブックは、IDとリアルタイム特徴量であるアルコール度数（ABV）を与えられたワインボトルの品質を予測するエンドポイントを作成します。

ノートブックの構成は次のとおりです：

1. 特徴量テーブルの準備。
2. Databricks Online Tableのセットアップ。
    * このノートブックはDatabricks Online Tablesを使用します。サポートされている機能のリストについては、Databricksのドキュメントを参照してください（[AWS](https://docs.databricks.com/aws/ja/machine-learning/feature-store/online-tables) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/machine-learning/feature-store/online-tables)）。  
3. モデルのトレーニングとデプロイ。
4. 自動特徴量ルックアップを使用したリアルタイムクエリの提供。
5. クリーンアップ。

### データセット

この例では、[Wine Quality Data Set](https://archive.ics.uci.edu/ml/datasets/wine+quality)を使用します。

### 要件

* サーバーレスコンピュート（[AWS](https://docs.databricks.com/aws/ja/compute/serverless) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/compute/serverless/)） (_推奨_)
* Databricks Runtime 14.2 for Machine Learning以上を実行するクラシックコンピュート。

<img src="https://docs.databricks.com/_static/images/machine-learning/feature-store/online-tables-nb-diagram.png"/>

In [0]:
%pip install databricks-sdk==0.41.0
%pip install databricks-feature-engineering==0.8.0
%pip install mlflow>=2.9.0
dbutils.library.restartPython()


## 特徴量テーブルの準備

`wine_id`だけでワインの品質を予測するエンドポイントを構築する必要があるとします。これには、エンドポイントが`wine_id`でワインの特徴を検索できるように、Feature Storeに保存された特徴量テーブルが必要です。このデモの目的のために、まずこの特徴量テーブルを自分で準備する必要があります。手順は次のとおりです：

1. 生データを読み込み、クリーンアップする。
2. 特徴量とラベルを分離する。
3. 特徴量を特徴量テーブルに保存する。

### 生データの読み込みとクリーンアップ

生データには11の特徴量と`quality`列を含む12列があります。`quality`列は3から8の範囲の整数です。目標は`quality`値を予測するモデルを構築することです。

In [0]:
raw_data_frame = spark.read.load("/databricks-datasets/wine-quality/winequality-red.csv",format="csv",sep=";",inferSchema="true",header="true" )
display(raw_data_frame.limit(10))

In [0]:
# 生データのサイズを確認します。
raw_data_frame.count(), len(raw_data_frame.columns)

生データにはいくつかの問題があります：
1. 列名にスペース（' '）が含まれており、これはFeature Storeと互換性がありません。
2. 後でFeature Storeで検索できるように、生データにIDを追加する必要があります。

次のセルでは、これらの問題に対処します。

In [0]:
from sklearn.preprocessing import MinMaxScaler
from pyspark.sql.functions import monotonically_increasing_id

# ID列を追加する関数
def addIdColumn(dataframe, id_column_name):
    columns = dataframe.columns
    new_df = dataframe.withColumn(id_column_name, monotonically_increasing_id())
    return new_df[[id_column_name] + columns]

# 列名を変更する関数
def renameColumns(df):
    renamed_df = df
    for column in df.columns:
        renamed_df = renamed_df.withColumnRenamed(column, column.replace(' ', '_'))
    return renamed_df

# 列名をFeature Storeに対応するように変更
renamed_df = renameColumns(raw_data_frame)

# ID列を追加
id_and_data = addIdColumn(renamed_df, 'wine_id')

display(id_and_data)

ワインが開封された後、アルコール度数（ABV）は時間とともに変化する変数であると仮定します。この値はオンライン推論のためにリアルタイム入力として提供されます。

次に、データを2つの部分に分割し、静的特徴を含む部分のみをFeature Storeに保存します。

In [0]:
# wine_idと静的特徴量
id_static_features = id_and_data.drop('alcohol', 'quality')

# wine_id、リアルタイム特徴量（alcohol）、ラベル（quality）
id_rt_feature_labels = id_and_data.select('wine_id', 'alcohol', 'quality')

### 特徴量テーブルを作成する

特徴量データ `id_static_features` を特徴量テーブルに保存します。

In [0]:
# カタログに対する `CREATE CATALOG` 権限が必要です。
# 必要に応じて、ここでカタログとスキーマ名を変更してください。
username = spark.sql("SELECT current_user()").first()["current_user()"]
username = username.split(".")[0]

# 既存のカタログを使います
catalog_name = "users"

# スキーマ名として使用するユーザー名を取得します。
schema_name = "takaaki_yayoi"

#spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
spark.sql(f"USE CATALOG {catalog_name}")
#spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalog_name}.{schema_name}")
spark.sql(f"USE SCHEMA {schema_name}")

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient

wine_table = f"{catalog_name}.{schema_name}.wine_static_features"
online_table_name = f"{catalog_name}.{schema_name}.wine_static_features_online"
fe = FeatureEngineeringClient()
fe.create_table(
    name=wine_table,
    primary_keys=["wine_id"],
    df=id_static_features,
    description="すべてのワインのIDと特徴量",
)

特徴量データは特徴量テーブルに保存されました。次のステップは、Databricksオンラインテーブルを設定することです。

## Databricksオンラインテーブルの設定

カタログエクスプローラーUI、Databricks SDK、またはRest APIからオンラインテーブルを作成できます。以下に、Databricks Python SDKを使用する手順を説明します。詳細については、Databricksのドキュメントを参照してください（[AWS](https://docs.databricks.com/en/machine-learning/feature-store/online-tables.html#create) | [Azure](https://learn.microsoft.com/azure/databricks/machine-learning/feature-store/online-tables#create)）。必要な権限については、権限に関する情報を参照してください（[AWS](https://docs.databricks.com/en/machine-learning/feature-store/online-tables.html#user-permissions) | [Azure](https://learn.microsoft.com/azure/databricks/machine-learning/feature-store/online-tables#user-permissions)）。

In [0]:
from pprint import pprint
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import OnlineTable, OnlineTableSpec, OnlineTableSpecTriggeredSchedulingPolicy

workspace = WorkspaceClient()

# オンラインテーブルを作成
spec = OnlineTableSpec(
  primary_key_columns = ["wine_id"],
  source_table_full_name = wine_table,
  run_triggered=OnlineTableSpecTriggeredSchedulingPolicy.from_dict({'triggered': 'true'}),
  perform_full_copy=True)

online_table = OnlineTable(name=online_table_name, spec=spec)

try:
  workspace.online_tables.create_and_wait(table=online_table)
  
except Exception as e:
  if "already exists" in str(e):
    print(f"オンラインテーブル {online_table_name} は既に存在します。再作成しません。")  
  else:
    raise e

pprint(workspace.online_tables.get(online_table_name))

## モデルのトレーニングとデプロイ

次に、Feature Storeの特徴量を使用して分類器をトレーニングします。主キーを指定するだけで、Feature Storeが必要な特徴量を取得します。

In [0]:
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import logging
import mlflow.sklearn

from databricks.feature_engineering import FeatureLookup

まず、`TrainingSet`を定義します。トレーニングセットは`feature_lookups`リストを受け取り、各アイテムはFeature Store内の特徴量テーブルからの特徴量を表します。この例では、`wine_id`をルックアップキーとして使用し、テーブル`online_feature_store_example.wine_features`からすべての特徴量を取得します。

In [0]:
# トレーニングセットを作成
training_set = fe.create_training_set(
    df=id_rt_feature_labels,
    label='quality',
    feature_lookups=[
        FeatureLookup(
            table_name=f"{catalog_name}.{schema_name}.wine_static_features",
            lookup_key="wine_id"
        )
    ],
    exclude_columns=['wine_id'],
)

# Feature Storeからトレーニングデータをロード
training_df = training_set.load_df()

display(training_df)

次のセルでは、RandomForestClassifierモデルをトレーニングします。

In [0]:
# トレーニングデータの特徴量とラベルを準備
X_train = training_df.drop('quality').toPandas()
y_train = training_df.select('quality').toPandas()

# モデルをトレーニング
model = RandomForestClassifier()
model.fit(X_train, y_train.values.ravel())

トレーニング済みモデルを`log_model`を使用して保存します。`log_model`は、モデルと特徴量（`training_set`を通じて）の間の系統情報も保存します。そのため、サービング中にモデルはルックアップキーだけで特徴量をどこから取得するかを自動的に認識します。

In [0]:
import mlflow

mlflow.set_registry_uri("databricks-uc")

registered_model_name = f"{catalog_name}.{schema_name}.wine_classifier"
fe.log_model(
    model=model,
    artifact_path="model",
    flavor=mlflow.sklearn,
    training_set=training_set,
    registered_model_name=registered_model_name
)

# 最新のモデルバージョンを取得
client = mlflow.tracking.MlflowClient()
versions = client.search_model_versions(f"name='{registered_model_name}'")
registered_model_version = max(int(v.version) for v in versions)

## 自動特徴量ルックアップを使用したリアルタイムクエリの提供

`log_model`を呼び出した後、新しいバージョンのモデルが保存されます。

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput

# エンドポイントを作成
endpoint_name = f"{username}_wine_classifier_endpoint"

try:
  status = workspace.serving_endpoints.create_and_wait(
    name=endpoint_name,
    config = EndpointCoreConfigInput(
      served_entities=[
        ServedEntityInput(
            entity_name=registered_model_name,
            entity_version=registered_model_version,
            scale_to_zero_enabled=True,
            workload_size="Small"
        )
      ]
    )
  )
  print(status)
except Exception as e:
  if "already exists" in str(e):
    print(f"エンドポイント {endpoint_name} は既に存在するため、作成しません。")
  else:
    raise e

## クエリを送信する

さて、ワインのボトルを開けて、ボトルから現在のABVを測定するセンサーがあるとします。モデルと自動特徴量ルックアップを使用したリアルタイムサービングを使用して、測定されたABV値をリアルタイム入力「alcohol」としてワインの品質を予測できます。

In [0]:
import mlflow.deployments

client = mlflow.deployments.get_deploy_client("databricks")
response = client.predict(
    endpoint=endpoint_name,
    inputs={
        "dataframe_records": [
            {"wine_id": 25, "alcohol": 7.9},
            {"wine_id": 25, "alcohol": 11.0},
            {"wine_id": 25, "alcohol": 27.9},
        ]
    },
)

pprint(response)

### リクエスト形式とAPIバージョンに関する注意事項

リクエストフォーマットの例は以下の通りです:
```
{"dataframe_split": {"index": [0, 1, 2], "columns": ["wine_id", "alcohol"], "data": [[25, 7.9], [25, 11.0], [25, 27.9]]}}
```

Databricksモデルサービングの詳細はこちらをご覧ください([AWS](https://docs.databricks.com/en/machine-learning/model-serving/index.html)|[Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/))。

## クリーンアップ

このノートブックで作成したリソースをクリーンアップするには、次の手順に従ってください:

1. Catalog ExplorerからDatabricksオンラインテーブルを削除します。  
  a. 左側のサイドバーで、**Catalog**をクリックします。  
  b. オンラインテーブルに移動します。  
  c. ケバブメニューから**Delete**を選択します。  
2. **Serving**タブからサービングエンドポイントを削除します。  
  a. 左側のサイドバーで、**Serving**をクリックします。  
  b. エンドポイントの名前をクリックします。  
  c. ケバブメニューから**Delete**を選択します。

In [0]:
#workspace.serving_endpoints.delete(name=endpoint_name)

In [0]:
#workspace.online_tables.delete(name=online_table_name)